In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import math

In [ ]:
user_id = 'your_userid_of_jobplanet'
password = 'your_password_of_jobplanet'

In [ ]:
driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(3)
driver.get('https://www.jobplanet.co.kr/companies?sort_by=review_culture_cache')

driver.find_element_by_xpath('/html/body/div[1]/div[1]/header/div/div[2]/a[1]').click()

driver.find_element_by_name('user[email]').send_keys(user_id)
driver.find_element_by_name('user[password]').send_keys(password)

driver.find_element_by_xpath('//*[@id="signInSignInCon"]/div[2]/div/section[2]/fieldset/button').click()

base_url = 'https://www.jobplanet.co.kr/companies?sort_by=review_culture_cache&page='
company_name_list = []
company_score_list = []
company_link_list = []

page_num = 1

for num in range(page_num):
    page_url = base_url + str(num)
    driver.get(page_url)
    driver.implicitly_wait(1)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    company_names = soup.select('#listCompanies > div > div.section_group > section > div > div > dl.content_col2_3.cominfo > dt > a')
    company_scores = soup.select('#listCompanies > div > div.section_group > section > div > div > dl.content_col2_4 > dd.gf_row > span')
    
    for name in company_names:
        company_name_list.append(name.text)
    for score in company_scores:
        company_score_list.append(score.text)
    for a_tag in company_names:
        company_link_list.append(re.findall('"(.+?)"', str(a_tag))[0])

In [ ]:
for i in range(len(company_name_list)):
    print(company_name_list[i], '\t', str(company_score_list[i]))

In [ ]:
my_url = 'https://www.jobplanet.co.kr'
good_text = []
bad_text = []

for link in company_link_list:
    one_url = my_url + link

    driver.get(one_url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    comment_nums = int(soup.select('#viewReviewsTitle > span')[0].text)

    current_url = driver.current_url

    for comment_page_num in range(math.ceil(comment_nums/5)):
        comment_url = current_url + '?page=' + str(comment_page_num + 1)

        driver.get(comment_url)
        driver.implicitly_wait(1)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        good_comments = soup.select('#viewReviewsList > div > div > div > section > div > div.ctbody_col2 > div > dl > dd:nth-of-type(1) > span')
        bad_commnets = soup.select('#viewReviewsList > div > div > div > section > div > div.ctbody_col2 > div > dl > dd:nth-of-type(2) > span')
        for comment in good_comments:
            good_text.append(comment.text)
        for comment in bad_commnets:
            bad_text.append(comment.text)

In [ ]:
all_good_text = ''
for text in good_text:
    all_good_text += text
    all_good_text += '\n'
    
all_bad_text = ''
for text in bad_text:
    all_bad_text += text
    all_bad_text += '\n'

In [ ]:
file = open("top20_good_text.txt", "w")
file.write(all_good_text)
file.close()

In [ ]:
file = open("top20_bad_text.txt", "w")
file.write(all_bad_text)
file.close()